In [ ]:
import os
import shutil
import random

# Chemin vers le dossier de données d'origine
data_dir = r'C:\Users\user\Desktop\deep_learning_cnn\data'

# Nouveaux chemins vers les dossiers
base_dir = 'data_test'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

# Création des nouveaux dossiers
os.makedirs(train_dir, exist_ok=True)
os.makedirs(validation_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Sous-dossiers pour les classes
classes = ['good', 'defective']

# Fonction pour déplacer les fichiers
def move_files(src_dir, dest_dir, split_ratio):
    filenames = os.listdir(src_dir)
    random.shuffle(filenames)
    
    split1 = int(split_ratio[0] * len(filenames))
    split2 = int((split_ratio[0] + split_ratio[1]) * len(filenames))

    train_files = filenames[:split1]
    validation_files = filenames[split1:split2]
    test_files = filenames[split2:]

    for fname in train_files:
        shutil.copy(os.path.join(src_dir, fname), os.path.join(dest_dir, 'train', os.path.basename(src_dir), fname))
    for fname in validation_files:
        shutil.copy(os.path.join(src_dir, fname), os.path.join(dest_dir, 'validation', os.path.basename(src_dir), fname))
    for fname in test_files:
        shutil.copy(os.path.join(src_dir, fname), os.path.join(dest_dir, 'test', os.path.basename(src_dir), fname))

# Répartition des données (80% entraînement, 10% validation, 10% test)
split_ratio = [0.8, 0.1, 0.1]

# Déplacement des fichiers pour chaque classe
for cls in classes:
    cls_src_dir = os.path.join(data_dir, cls)
    for split in ['train', 'validation', 'test']:
        os.makedirs(os.path.join(base_dir, split, cls), exist_ok=True)
    move_files(cls_src_dir, base_dir, split_ratio)

print("Réorganisation des fichiers terminée.")


In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Nadam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image

In [29]:
# Chemins vers les jeux de données
base_dir = r'C:\Users\user\Desktop\deep_learning_cnn\data_test'
train_data_dir = os.path.join(base_dir, 'train')
validation_data_dir = os.path.join(base_dir, 'validation')
test_data_dir = os.path.join(base_dir, 'test')
#model_path = r'C:\Users\user\Desktop\deep_learning_cnn\model\model.h5'

In [11]:
# Paramètres du modèle
img_width, img_height = 224, 224
batch_size = 32
epochs = 50
num_classes = 1  # Binaire : bon ou défectueux

In [12]:
# Préparation des données
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

In [13]:
test_datagen = ImageDataGenerator(rescale=1. / 255)

In [14]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary'
)

Found 1484 images belonging to 2 classes.


In [18]:
validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary'
)

Found 186 images belonging to 2 classes.


In [15]:
# Création du modèle
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='sigmoid')
])

c:\Users\user\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [16]:
# Compilation du modèle
optimizer = Nadam(clipnorm=1.0)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [19]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    epochs=epochs
)

c:\Users\user\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


46/46 ━━━━━━━━━━━━━━━━━━━━ 166s 3s/step - accuracy: 0.5342 - loss: 1.4916 - val_accuracy: 0.6375 - val_loss: 0.6312


In [30]:
# Sauvegarde du modèle
#model.save(model_path)

In [31]:
model.save(r'C:\Users\user\Desktop\deep_learning_cnn\model/model.keras', save_format='keras_v3')


In [52]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.optimizers import Nadam
from PIL import Image

# Chemin vers le modèle sauvegardé
model_path = r'C:\Users\user\Desktop\deep_learning_cnn\model\model.keras'  # ou 'model.keras'

# Fonction de test du modèle sur une image
def test_model(model_path, image_path):
    try:
        # Charger le modèle sauvegardé
        model = tf.keras.models.load_model(model_path)
        optimizer = Nadam(clipnorm=1.0)
        model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

        # Charger et préparer l'image
        img = Image.open(image_path)
        img = img.resize((224, 224))
        img_array = np.array(img) / 255.0
        img_input = np.expand_dims(img_array, axis=0)

        # Effectuer la prédiction
        predictions = model.predict(img_input)
        probability = predictions[0][0]

        # Déterminer le résultat basé sur la probabilité
        if probability > 0.5:
            result = "Good"
        else:
            result = "Defective"

        return result, probability
    except Exception as e:
        print(f"Erreur lors du traitement de l'image {image_path}: {e}")
        return None, None

# Chemin vers le répertoire des images de test
test_images_dir = r'C:\Users\user\Desktop\deep_learning_cnn\data_test\test'

# Itération sur les sous-dossiers 'good' et 'defective'
for cls in ['good', 'defective']:
    cls_dir = os.path.join(test_images_dir, cls)
    if os.path.exists(cls_dir):
        for fname in os.listdir(cls_dir):
            image_path = os.path.join(cls_dir, fname)
            result, probability = test_model(model_path, image_path)
            if result is not None and probability is not None:
                print(f"Résultat de la prédiction pour {image_path}: {result} avec une probabilité de {probability:.2f}")
            else:
                print(f"Échec de la prédiction pour {image_path}.")
    else:
        print(f"Le répertoire {cls_dir} n'existe pas.")


c:\Users\user\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 12 variables whereas the saved optimizer has 23 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
Résultat de la prédiction pour C:\Users\user\Desktop\Project_diaaa\path_to_your_new_data_directory\test\good\good (114).jpg: Good avec une probabilité de 0.71
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
Résultat de la prédiction pour C:\Users\user\Desktop\Project_diaaa\path_to_your_new_data_directory\test\good\good (115).jpg: Good avec une probabilité de 0.51
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
Résultat de la prédiction pour C:\Users\user\Desktop\Project_diaaa\path_to_your_new_data_directory\test\good\good (120).jpg: Good avec une probabilité de 0.94
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
Résultat de la prédiction pour C:\Users\user\Desktop\Project_diaaa\path_to_your_new_data_directory\test\good\good (122).jpg: Good avec une probabilité de 0.83


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
Résultat de la prédiction pour C:\Users\user\Desktop\Project_diaaa\path_to_your_new_data_directory\test\good\good (130).jpg: Good avec une probabilité de 0.97


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
Résultat de la prédiction pour C:\Users\user\Desktop\Project_diaaa\path_to_your_new_data_directory\test\good\good (132).jpg: Good avec une probabilité de 0.99
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
Résultat de la prédiction pour C:\Users\user\Desktop\Project_diaaa\path_to_your_new_data_directory\test\good\good (138).jpg: Defective avec une probabilité de 0.36
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
Résultat de la prédiction pour C:\Users\user\Desktop\Project_diaaa\path_to_your_new_data_directory\test\good\good (153).jpg: Good avec une probabilité de 0.72
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
Résultat de la prédiction pour C:\Users\user\Desktop\Project_diaaa\path_to_your_new_data_directory\test\good\good (157).jpg: Good avec une probabilité de 0.72
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
Résultat de la prédiction pour C:\Users\user\Desktop\Project_diaaa\path_to_your_new_data_directory\test\good\good (162).jpg: Good avec une probabilité de 0.84
1/1 ━━━━━━